## 实战tensorflow2.0 搭建一个ANN，全连接神经网络

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import functools
import warnings
warnings.filterwarnings('ignore')

E:\Anaconda5.2\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda5.2\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda5.2\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda5.2\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of 

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TRAIN_DATA_URL)

In [3]:
np.set_printoptions(precision=3, suppress=True)  ## 控制文件中小数点的位数
## np.set_printoptions(precision=3, suppress=True)
# 参数：
# precision: 保留几位小数，后面不会补0
# supress: 对很大/小的数不使用科学计数法 (true)
# formatter: 强制格式化，后面会补0

In [4]:
type(train_file_path)

str

In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=5,
    label_name=LABEL_COLUMN,
    na_value="?",
    num_epochs=1,
    ignore_errors=True,
    **kwargs)
    return dataset

In [7]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [8]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}:{}".format(key, value.numpy()))

In [9]:
show_batch(raw_train_data)

sex                 :[b'male' b'male' b'female' b'female' b'male']
age                 :[39. 18. 39.  4. 28.]
n_siblings_spouses  :[0 1 1 0 0]
parch               :[0 0 5 1 0]
fare                :[26.     6.496 31.275 13.417  8.05 ]
class               :[b'Second' b'Third' b'Third' b'Third' b'Third']
deck                :[b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         :[b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               :[b'y' b'n' b'n' b'n' b'y']


In [10]:
## 如果文件第一行没有列名，可以手动加列名
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 :[b'male' b'male' b'male' b'female' b'male']
age                 :[51. 21. 24. 28. 28.]
n_siblings_spouses  :[0 0 0 1 0]
parch               :[0 0 0 0 0]
fare                :[ 26.55    7.796  79.2   146.521   7.896]
class               :[b'First' b'Third' b'First' b'First' b'Third']
deck                :[b'E' b'unknown' b'B' b'B' b'unknown']
embark_town         :[b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton']
alone               :[b'y' b'y' b'y' b'n' b'y']


In [11]:
## select_columns 选中固定的列
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 :[42. 19. 28. 50. 34.]
n_siblings_spouses  :[0 0 0 1 0]
class               :[b'Third' b'Third' b'First' b'First' b'Second']
deck                :[b'unknown' b'unknown' b'unknown' b'C' b'unknown']
alone               :[b'y' b'y' b'y' b'n' b'y']


In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS, column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 :[19. 24. 30. 44. 28.]
n_siblings_spouses  :[0. 1. 0. 0. 1.]
parch               :[0. 2. 0. 1. 0.]
fare                :[ 7.896 65.     7.896 57.979 16.1  ]


In [14]:
example_batch, labels_batch = next(iter(temp_dataset)) 

## Data preprocessing

## 常用的几种数据预处理方式
### 1、sklearn+pandas+numpy
### 2、tensorflow内置处理接口
### 在模型内部进行预处理的主要优点是，在导出模型时，它包括预处理。 这样，您可以将原始数据直接传递到模型。

In [15]:
# x = tf.constant([1, 4])
# y = tf.constant([2, 5])
# z = tf.constant([3, 6])
# tf.stack([x, y, z])

In [16]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [20]:
packed_dataset = temp_dataset.map(pack)
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[19.     0.     0.     7.896]
 [24.     1.     2.    65.   ]
 [30.     0.     0.     7.896]
 [44.     0.     1.    57.979]
 [28.     1.     0.    16.1  ]]

[0 1 0 1 1]


In [21]:
example_batch, labels_batch = next(iter(temp_dataset))

In [22]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [23]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']
packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
paced_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [24]:
show_batch(packed_train_data)

sex                 :[b'male' b'male' b'female' b'female' b'male']
class               :[b'Second' b'Third' b'Third' b'Third' b'Third']
deck                :[b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         :[b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               :[b'y' b'n' b'n' b'n' b'y']
numeric             :[[39.     0.     0.    26.   ]
 [18.     1.     0.     6.496]
 [39.     1.     5.    31.275]
 [ 4.     0.     1.    13.417]
 [28.     0.     0.     8.05 ]]


In [25]:
example_batch, labels_batch = next(iter(packed_train_data)) 

## Data Normalization

In [26]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [27]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [28]:
def normalize_numeric_data(data, mean, std):
    return(data - mean) / std

In [29]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001B1C9C548C8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [30]:
example_batch['numeric']

<tf.Tensor: id=589, shape=(5, 4), dtype=float32, numpy=
array([[39.   ,  0.   ,  0.   , 26.   ],
       [18.   ,  1.   ,  0.   ,  6.496],
       [39.   ,  1.   ,  5.   , 31.275],
       [ 4.   ,  0.   ,  1.   , 13.417],
       [28.   ,  0.   ,  0.   ,  8.05 ]], dtype=float32)>

In [31]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

Instructions for updating:
Use `tf.cast` instead.


array([[ 0.749, -0.474, -0.479, -0.154],
       [-0.93 ,  0.395, -0.479, -0.511],
       [ 0.749,  0.395,  5.827, -0.057],
       [-2.049, -0.474,  0.782, -0.384],
       [-0.13 , -0.474, -0.479, -0.482]], dtype=float32)

## Categorical data
### Use the tf.feature_column API to create a collection with a tf.feature_column.indicator_column for each categorical column.

In [32]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [33]:
tf.feature_column.indicator_column??

In [34]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [35]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


## Combined preprocessing layer

In [36]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [37]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     1.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.749 -0.474
 -0.479 -0.154  1.     0.   ]


## Build the model

In [38]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
                          
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer='adam',
             metrics=['accuracy'])

In [39]:
train_data = packed_train_data.shuffle(500)
test_data = paced_test_data

In [40]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 3s 26ms/step - loss: 0.5102 - accuracy: 0.6701
Epoch 2/20
126/126 [==============================] - 3s 20ms/step - loss: 0.4286 - accuracy: 0.8131
Epoch 3/20
126/126 [==============================] - 3s 20ms/step - loss: 0.4058 - accuracy: 0.8268
Epoch 4/20
126/126 [==============================] - 3s 20ms/step - loss: 0.3928 - accuracy: 0.8354
Epoch 5/20
126/126 [==============================] - 3s 20ms/step - loss: 0.3828 - accuracy: 0.8387
Epoch 6/20
126/126 [==============================] - 3s 20ms/step - loss: 0.3740 - accuracy: 0.8376
Epoch 7/20
126/126 [==============================] - 3s 20ms/step - loss: 0.3661 - accuracy: 0.8338
Epoch 8/20
126/126 [==============================] - 3s 20ms/step - loss: 0.3586 - accuracy: 0.8344
Epoch 9/20
126/126 [==============================] - 3s 21ms/step - loss: 0.3522 - accuracy: 0.8286 0s - loss: 0.3587 - accuracy
Epoch 10/20
126/126 [==============================] - 3s 20ms

In [41]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

    126/Unknown - 2s 17ms/step - loss: 0.2745 - accuracy: 0.8740

Test Loss 0.27447100038388894, Test Accuracy 0.8740031719207764


In [42]:
# !pip install mlxtend

In [43]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix